In [1]:
from pymilvus import connections, db

# milvus_conn = connections.connect(host="127.0.0.1", port=19530)

database_name = "docs"
# db.using_database("book")
milvus_conn = connections.connect(
    host="127.0.0.1",
    port="19530",
    # db_name="book"
)

if database_name not in db.list_database():
    db.create_database(database_name)

db.using_database(database_name)
db.list_database()


['default', 'docs']

In [4]:
import ollama
import chromadb

documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

client = chromadb.Client()

# 嘗試取得已存在的 collection，如果不存在則創建新的
try:
    collection = client.create_collection(name="docs")
except Exception as e:
    if "Collection docs already exists" in str(e):
        collection = client.get_collection(name="docs")
    else:
        raise e

# 檢查每個文件 ID 是否已存在，如果存在則跳過新增操作
existing_docs = collection.get()
existing_ids = set(existing_docs['ids'])

# store each document in a vector embedding database
for i, d in enumerate(documents):
    if str(i) in existing_ids:
        print(f"ID {i} already exists, skipping.")
        continue

    response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
    embedding = response["embedding"]
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        documents=[d]
    )

ID 0 already exists, skipping.
ID 1 already exists, skipping.
ID 2 already exists, skipping.
ID 3 already exists, skipping.
ID 4 already exists, skipping.
ID 5 already exists, skipping.


In [8]:
# an example prompt
prompt = "What animals are llamas related to?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=1
)
data = results['documents'][0][0]

In [9]:
ollama.pull(model="llama2")


Llamas are members of the camelid family, which means they are closely related to two other animals: vicuñas and camels. All three species belong to the same taxonomic family (Camelidae) and share many similarities in their physical characteristics and behaviors. Specifically, llamas are most closely related to vicuñas, with which they share a common ancestor that lived around 20-30 million years ago. Camels are also members of this family, but they are more distantly related to llamas than vicuñas.


In [10]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama2",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])


Llamas are related to several other animals within the camelid family. Specifically, they are most closely related to vicuñas and camels. All three of these animals belong to the family Camelidae and share many similarities in their physical characteristics and behavior. Here are some key similarities between llamas, vicuñas, and camels:

1. Physical appearance: Llamas, vicuñas, and camels all have long necks, legs, and ears, as well as a distinctive hump on their backs. They also have thick, coarse coats that help them survive in hot, dry climates.
2. Diet: These animals are all herbivores and feed primarily on grasses, leaves, and other vegetation.
3. Habitat: Llamas, vicuñas, and camels are all found in the Andean region of South America, where they inhabit high-altitude grasslands and deserts.
4. Social behavior: These animals are social creatures and often live in herds. They communicate with each other through a variety of vocalizations, including grunts, bleats, and whistles.
5